# Nonideal Capacitor Behavior

Real components require physical connections to other circuit components, and these connections result in parasitic behaviors.  Parasitic behaviors are any circuit behaviors that are not part of the intended schematic.  In this notebook, we study the effects of parasitic inductance on capacitors.  The connection into the rest of the circuit implies a current loop, and that current loop has an inductance associated with it.  This parasitic inductance is in series with the intended capacitance.  The parasitic inductance limits the operational frequency range of the capacitor.

In [7]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
pi = np.pi
sqrt = np.sqrt

from numpy.linalg import solve as linsolve

from matplotlib import rc
#rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
## for Palatino and other serif fonts use:
#rc('font',**{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)

# from ipywidgets import jslink, FloatSlider, IntSlider, VBox, RadioButtons, Checkbox, interactive, Text
# from IPython.display import display, HTML
# style = {'description_width': 'initial'}

from IPython.display import Image

def legend_alpha(leg, alpha=0.25):
    leg.get_frame().set_alpha(alpha)
    return leg

def legend_linewidth(leg, width=2.0):
    for ihandle in leg.legendHandles:
        ihandle.set_linewidth(width)
    return leg

default_save_formats = ['png', 'pdf']
def savefigure(fig, filename, path='figs', **kwargs):
    from os.path import join as os_path_join
    pathfilename = os_path_join(path, filename)
    formats = kwargs.get('formats', default_save_formats)
    transparent = kwargs.get('transparent', False)
    for format_i in formats:
        fig.savefig(pathfilename + '.' + format_i,
                    transparent=transparent)

def printp(X, L):
    print(L, " = ", np.abs(X), " angle ", np.angle(X)*180/pi, " deg")

def printr(X, L):
    print(L, " = ", X)
    
def printph(X, L):
    printp(X, L)
    printr(X, L)
    
def printsc(X, L):
    print(L, " = ", X)
    
def ph(angle_deg):
    angle_rad = pi*angle_deg/180.0;
    return (np.cos(angle_rad) + 1j*np.sin(angle_rad))

def flatten(L):
    if hasattr(L, '__iter__'):
        if len(L) == 0:
            return []
        first, rest = L[0], L[1:]
        return flatten(first) + flatten(rest)
    else:
        return [L, ]
    
def inv_eq(*args):
    sum_of_inverses = 0
    for Z_ in args:
        sum_of_inverses += 1./Z_ 
    Xeq = 1./sum_of_inverses
    return Xeq

def parallelL(*args):
    return inv_eq(*args)

def seriesC(*args):
    return inv_eq(*args)

def parallelZ(*args):
    return inv_eq(*args)

def ph2vec(phasor_):
    return np.array([np.real(phasor_), np.imag(phasor_)])

def norm(v_):
    return np.sqrt(np.inner(v_, v_))

def db(X):
    return 20.0*np.log10(np.abs(X))

def phase_deg(X):
    return 180*np.unwrap(np.angle(X))/np.pi


In [8]:
def zerocross(t, v, vref, **kwargs):
    """
    Get the indices/time where the voltage crosses the reference
    voltage.

    **kwargs:

    slope - which direction to detect the crossings. Valid values are
    positive, negative, both, rise, fall (default: both)

    gettrace - return a trace containing a 0 for samples with no
    transition, 1 for samples with a positive slope transition, and -1
    for samples with a negative slope transition.  If the slope
    detection is set to only catch positive or negative, the other
    will not be present in the trace either. (default: False)

    mode - determines how the actual crossing time is reported.  If
    mode == 0 (default): calculates crossing by a linear interpolation
    of the 2D coordinates.  If mode == 1: returns the midpoint between
    the time samples as the crossing.  Otherwise, return the time
    sample immediately before the crossing.
    """

    ###############################################################
    # Keep this for debugging code using the older option names.
    for argkey in ['CrossingType', 'GetTrace', 'Mode']:
        if argkey in kwargs:
            print("%s has been removed as a kwarg.  Check for the updated argument name in tracemath.py.")
            return None
    ###############################################################

    ###############################################################
    # The following sets default values and replaces any that are
    # specified when the function is called.
    settings = dict(slope='both', gettrace=False, getIndices=False, mode=0)
    for key, val in kwargs.items():
        settings[key] = val
    ###############################################################

    N = len(t)
    zc = np.zeros(N)  # zero crossings
    for ii in range(0, N-1):
        if (v[ii+1] >= vref) and (v[ii] < vref):  # positive transition
            zc[ii] = 1
        if (v[ii+1] < vref) and (v[ii] >= vref):  # negative transition
            zc[ii] = -1

    if settings['gettrace']:
        return zc

    # Otherwise get the crossing times.
    # Rise and fall are the same as positive and negative, respectively.
    ct = settings['slope'].lower()
    bZCArray = {
        'both': lambda XA: XA != 0,
        'positive': lambda XA: XA > 0,
        'rise': lambda XA: XA > 0,
        'negative': lambda XA: XA < 0,
        'fall': lambda XA: XA < 0 }[ct](zc)

    if settings['getIndices']:
        return np.where(bZCArray)[0]


    # I have a couple of calculation options, but the linear interpolation is
    # the most accurate that I have built in so far.  A quadratic search is
    # probably overkill.
    ZCTime = np.array([])
    Mode = settings['mode']
    if Mode == 0:
        ZCTimeList = []
        for ii in range(0, N-1):
            if bZCArray[ii]:
                # Calculate the linear interpolated crossing time.
                lam0 = (vref-v[ii+1])/(v[ii] - v[ii+1])
                tZeroCrossing = lam0 * t[ii] + (1-lam0) * t[ii+1]
                ZCTimeList.append(tZeroCrossing)
        ZCTime = np.array(ZCTimeList)
    elif Mode == 1:
        ZCTime = np.array([0.5*(t[ii]+t[ii+1]) for ii in range(0,N-1) if bZCArray[ii]])
    else:
        ZCTime = np.array([t[ii] for ii in range(0,N-1) if bZCArray[ii]])

    # Build a crossing file.
    if 'filename' in kwargs:
        fmt = kwargs.get('format', '%.12g')
        filename = kwargs['filename']
        s = '\n'.join([fmt % z for z in ZCTime]) + '\n'
        with open(filename, 'w') as f:
            f.write(s)
    return ZCTime

def nearest_index(a, v):                                               
    """                                                                
    Act similar to numpy.searchsorted, but find the nearest point      
    rather than using a before or after approach.                      
                                                                       
    a - is the full vector                                             
    v - is the value to search on.                                     
    """                                                                
    if type(v) in (list, tuple, np.ndarray):                           
        inearest = []                                                  
        for v_ in v:                                                   
            inearest.append(nearest_index(a, v_))                      
        return inearest                                                
    else:                                                              
        # The real calculation.                                        
        return np.argmin(np.abs(np.array(a)-v))                        
                                                                       
                                                                       
def marker_fcn(a, v, y):                                               
    """                                                                
    Use nearest_index to determine the closest indices.  Then, output  
    the y values associated with these indices like a marker on an     
    oscilloscope or spectrum analyzer.                                 
    """                                                                
    il = nearest_index(a, v)                                           
    return np.array(y)[il]                                             

In [9]:
Rp = 0.1
Lp = 15e-9
C = 350e-9

f = np.logspace(1,10,501)
w = 2*np.pi*f
jw = 1j*2*np.pi*f

Zc = Rp + jw*Lp + 1./(jw*C)
Zs = 50
Zl = 50

Zload_eq = parallelZ(Zl, Zc)

# G = Vo / Vs
G = Zload_eq / (Zload_eq + Zs)

In [10]:
def Cest(fmarker_, G_ = G, f_ = f):
    Gabs = np.abs(G_)
    #print("frequency of C: ", f_[nearest_index(f_, fmarker_)])
    Gabs_ = marker_fcn(f_, fmarker_, Gabs)
    Cest = 1./(50*np.pi*fmarker_) * np.sqrt(1./(4*(Gabs_**2)) - 1)
    return Cest

def Cest_simple(fmarker_, G_ = G, f_ = f):
    Gabs = np.abs(G_)
    #print("frequency of C: ", f_[nearest_index(f_, fmarker_)])
    Gabs_ = marker_fcn(f_, fmarker_, Gabs)
    Cest = 1./(2*np.pi*fmarker_*Zs*Gabs_)
    return Cest
    
def Lest(fmarker_, G_ = G, f_ = f):
    Gabs = np.abs(G_)
    #print("frequency of L: ", f_[nearest_index(f_, fmarker_)])
    Gabs_ = marker_fcn(f_, fmarker_, Gabs)
    Lest = 25./(2*np.pi*fmarker_*np.sqrt((1./(2*Gabs_))**2 - 1))
    return Lest

def Lest_simple(fmarker_, G_ = G, f_ = f):
    Gabs = np.abs(G_)
    #print("frequency of L: ", f_[nearest_index(f_, fmarker_)])
    Gabs_ = marker_fcn(f_, fmarker_, Gabs)
    Lest = Gabs_*Zs/(2*np.pi*fmarker_)
    return Lest


In [11]:
fig = plt.figure()
ax = fig.add_subplot(2,1,1)
ax2 = fig.add_subplot(2,1,2, sharex=ax)

ax.loglog(f, np.abs(Zc), label='Nonideal Response')
ax.grid(True)
ax.set_title('Nonideal Capacitor Impedance')
ax.set_ylabel('Magnitude (Ohms)')

ax.set_ylim(1e-1/2,1e5)
ax2.semilogx(f, phase_deg(Zc))
ax2.grid(True)
ax2.set_ylabel('Phase (deg)')
ax2.set_xlabel('Frequency (Hz)')

ax2.set_yticks([-90, -45, 0, 45, 90])

savefigure(fig, 'nonideal_cap_impedance', path='.')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)

ax.loglog(f, np.abs(Zc), label='Nonideal Response')
ax.grid(True)
ax.set_title('Nonideal Capacitor Impedance')
ax.set_ylabel('Magnitude (Ohms)')
ax.set_xlabel('Frequency (Hz)')
ax.set_ylim(1e-1/2,1e5)
ax.loglog(f, np.abs(jw*Lp), label='Parasitive Inductance', alpha=0.5)
ax.loglog(f, np.abs(1./(jw*C)), label='Ideal Capacitance', alpha=0.5)
ax.loglog(f, np.abs(Rp+jw*0), label='ESR', alpha=0.5)
leg = ax.legend(loc='best')
legend_alpha(leg, 0)
legend_linewidth(leg, 1.5)
savefigure(fig, 'nonideal_cap_impedance_full', path='.')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
fig = plt.figure()
ax = fig.add_subplot(2,1,1)
ax2 = fig.add_subplot(2,1,2, sharex=ax)

ax.semilogx(f, db(G))
ax.grid(True)
ax2.semilogx(f, phase_deg(G))
ax2.grid(True)
ax2.set_xlabel('Frequency (Hz)')
ax.set_ylabel('Magnitude (dB)')
ax2.set_ylabel('Phase (deg)')
ax.set_title('Gain (Vo/Vs)')
ax2.set_yticks([-90, -45, 0, 45, 90])

savefigure(fig, 'nonideal_cap_gain_response', path='.')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
# Calculate particular frequencies
G0 = db(G[0])
G3db = G0 - 3

f3db = zerocross(f, db(G), G3db)
print("3dB Points: ", f3db)


3dB Points:  [1.80727595e+04 2.67077459e+08]


In [15]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)

ax.semilogx(f, db(G))
ax.grid(True)

ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('Magnitude (dB)')
ax.set_title('Gain (Vo/Vs)')

ax.axvline(f3db[0], linestyle='--')
ax.axvline(f3db[1], linestyle='--')

savefigure(fig, 'nonideal_cap_gain_response_markers', path='.')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
fig = plt.figure()
axc = fig.add_subplot(2,1,1)
axl = fig.add_subplot(2,1,2, sharex=axc)

Cev = Cest(f)
Cesv = Cest_simple(f)
Lev = Lest(f)
Lesv = Lest_simple(f)

axc.semilogx(f, Cev*1e9, label='Detailed Exp', linewidth=2)
axc.semilogx(f, Cesv*1e9, label='Simplified Exp')
axl.semilogx(f, Lev*1e9, label='Detailed Exp', linewidth=2)
axl.semilogx(f, Lesv*1e9, label='Simplified Exp')

axl.set_xlabel('Frequency (Hz)')
axc.set_ylabel('Capacitance (nF)')
axc.set_ylim(0.1*C*1e9, 3*C*1e9)
axc.axhline(C*1e9, linestyle='--', color='k', alpha=0.5)
axc.grid(True)
axl.set_ylabel('Inductance (nF)')
axl.set_ylim(0.1*Lp*1e9, 3*Lp*1e9)
axl.grid(True)
axl.axhline(Lp*1e9, linestyle='--', color='k', alpha=0.5)

leg = axc.legend(loc='best')
legend_alpha(leg, 0)
legend_linewidth(leg, 1.5)

savefigure(fig, 'c_and_lp_estimates', path='.')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …